In [1]:
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import math

In [2]:
key = 'oUCrKrbyUxcUHhMTDpWdajt3NeLpYGDXVRwd9WoeJQZMvfn3mDCIMb6S7BeuYV9ETrEdeynPovWdIQ%2BnV1sdNg%3D%3D'
url = 'http://apis.data.go.kr/1192000/select0030List/getselect0030List'
dec_key ='oUCrKrbyUxcUHhMTDpWdajt3NeLpYGDXVRwd9WoeJQZMvfn3mDCIMb6S7BeuYV9ETrEdeynPovWdIQ+nV1sdNg=='

name_list = ['위판일자','수산물표준코드명','위판중량']
data = []
cdf = pd.DataFrame()

##응답 메시지 명세

resultCode	결과코드	

resultMsg	결과메세지	

pageNo	페이지 수	 

numOfRows	한 페이지 결과 수	

totalCount	데이터 총 개수	

csmtDe	위판일자	

mxtrCode	산지조합코드	

mxtrNm	산지조합명	

csmtmktCode	위판장코드	

csmtmktNm	위판장명	

fshrCode	어업코드	

fshrNm	어업명	

prchasNo	매수번호	

prchasSn	매수일련번호	

mprcStdCode	수산물표준코드	

mprcStdCodeNm	수산물표준코드명	

csmtQy	위판수량	

csmtWt	위판중량	

csmtUnqt	위판단량	

csmtUntpc	위판단가	

csmtAmount	위판금액	

kdfshSttusCode	어종상태코드	

kdfshSttusNm	어종상태명	

goodsStndrdCode	상품규격코드

goodsStndrdNm	상품규격명	

goodsUnitCode	삼품단위코드	

goodsUnitNm	상품단위명	

orgplceSeCode	원산지구분코드	

orgplceSeNm	원산지명	


    

In [3]:
fishcode = 610802
fish = '고등어'
date = datetime.today()

In [4]:
def makedf(rows):
    rowList = []
    nameList = []
    columnList = []

    rowsLen = len(rows)
    for i in range(0, rowsLen):
        columns = rows[i].find_all()
        
        columnsLen = len(columns)
        for j in range(0, columnsLen):
            # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없다.)
            if i == 0:
                nameList.append(columns[j].name)
            # 컬럼값은 모든 행의 값을 저장해야한다.    
            eachColumn = columns[j].text
            columnList.append(eachColumn)
        rowList.append(columnList)
        columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)
        
    result = pd.DataFrame(rowList, columns=nameList)
    result.head()
    return(result)

In [5]:
params ={'serviceKey' : dec_key, 'numOfRows' : '100', 'pageNo' : '1', 'type' : 'xml', 'baseDt' : date, 'mprcStdCodeNm' : fish, }
response = requests.get(url, params=params).text.encode('utf-8')
xmlobj = BeautifulSoup(response, 'lxml-xml')
item_count = len(xmlobj.find_all('item'))
if xmlobj.find('resultCode').text == '00':
    rows = xmlobj.findAll('item')
    today_df = makedf(rows)
    tdf = today_df[['csmtDe','csmtWt']]
    cdf = pd.concat([cdf, tdf], axis=0)
elif xmlobj.find('resultCode').text != '00':
    print('error code :',xmlobj.find('resultCode').text)
    print('error message :',xmlobj.find('resultMsg').text)
else : 
    pass


error code : 03
error message : 조회된 데이터가 없습니다.


In [6]:
page_nm = 2
while item_count :
    if item_count < 100 :
        break
    else : 
        params ={'serviceKey' : dec_key, 'numOfRows' : '100', 'pageNo' : str(page_nm), 'type' : 'xml', 'baseDt' : date, 'mprcStdCodeNm' : fish }
        response = requests.get(url,params=params).text.encode('utf-8')
        xmlobj = BeautifulSoup(response, 'lxml-xml')
        item_count = len(xmlobj.find_all('item'))
        if item_count>=100:
            page_nm+=1
        try:
            if xmlobj.find('resultCode').text=='00' : 
                rows = xmlobj.findAll('item')
                today_df = makedf(rows)
                tdf = today_df[['csmtDe','csmtWt']]
                cdf = pd.concat([cdf,tdf],axis=0) 
        except : 
            if xmlobj.find('resultCode').text != '00' :
                print(xmlobj.find('resultMsg').text)
                break
            else:
                print('종료')
                break


In [7]:
if len(cdf) != 0 :
    cdf.columns = ['위판일자','위판중량']
    cdf['위판일자'] = pd.to_datetime(cdf['위판일자'])
    cdf['위판중량'] = pd.to_numeric(cdf['위판중량'])
    cdf = cdf.groupby(cdf['위판일자'].dt.strftime('%Y-%m-%d')).sum()
    cdf['품종']=fish
    cdf = cdf[['품종','위판중량']]
cdf

Empty DataFrame
Columns: []
Index: []